In [ ]:
# https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/

In [2]:
import re
import pickle
import random

import numpy as np
import pandas as pd
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

In [3]:
pickle_in = open("plots_text.pickle","rb")
movie_plots = pickle.load(pickle_in)

# count of movie plot summaries
len(movie_plots)

500

In [4]:
movie_plots[499]

"a village in rural thailand is celebrating loy krathong, when the festivities are disrupted by the descent of a spaceship. ray beams are fired from the craft and all the village's women find they are suddenly pregnant. only a few hours later the women give birth. the alien offspring have the power to kill by just staring and they have an insatiable appetite for raw meat."

In [5]:
# sample random summaries
random.sample(movie_plots, 6)

['little red riding hood discovers that the big bad wolf has disguised himself as granny, just as the ax-wielding woodsman bursts through the window, startling everyone. the police quickly arrive, and red, wolf, granny, and the woodsman are questioned by detective nicky flippers about the events leading up the incident. red explains that she was delivering goodies for her grandmother when she discovered a threat from the mysterious goodie bandit. hoping to save her granny’s recipes, she embarked upon a journey to take them to the top of a nearby mountain where her granny lives. on her way, she encountered the wolf, who asked her a series of suspicious questions. she managed to escape, and eventually reached her granny’s house; however she found the wolf already waiting in ambush. what at first seems to be an open-and-shut case becomes confused though, once it is learned that the wolf is an investigative reporter. he reveals that he was searching for a lead on the identity of the goodie

In [6]:
# clean text
movie_plots = [re.sub("[^a-z' ]", "", i) for i in movie_plots]

In [7]:
a = [1,2,3,4,5]

a[2:5]

[3, 4, 5]

In [8]:
movie_plots[1][0:5]

'chine'

In [9]:
movie_plots[1].split()[0:5]

['chinese', 'exorcist', 'oneeyebrow', 'priest', 'leads']

In [10]:
# sample random summaries
random.sample(movie_plots, 6)

["a hollywood wolf makes a pass at a cute movie usherette gets slapped in the face then settles down for the show but his juices get flowing again when the feature  comes on and he's inflamed by the hot romantic scenes between bogey gocart and laurie becool",
 "plot the film begins with an aging man named vengadam  being released from jail in  he is only being escorted for two days  back to his hometown of kanchipuram he is being transported under the custody of two policemen on a bus from as the journey takes place vengadam recalls his past symbolically as several events that occur in the bus  remind him of his past vengadam is a silk weaver in the town of kanchivaram and has just recently got married  he had vowed once that he will only wed a woman wearing a silk saree but had to settle for normalcy as he unable to garner enough savings to buy one it is then revealed through his life proceedings that silk weavers are only paid a very low sum  rupees for each silk saree woven this mea

In [115]:
def create_seq(text,seq_len=5):
    sequence = []
    for i in text:
        for j in range(0,len(i.split())):
            seq = i.split()[j:j+seq_len]
            if len(seq) < seq_len:
                break
            sequence.append(seq)
    return(sequence)

In [116]:
test = create_seq(movie_plots,4)

In [117]:
test

[['barry', 'is', 'a', 'private'],
 ['is', 'a', 'private', 'with'],
 ['a', 'private', 'with', 'the'],
 ['private', 'with', 'the', 'st'],
 ['with', 'the', 'st', 'airborne'],
 ['the', 'st', 'airborne', 'division'],
 ['st', 'airborne', 'division', 'of'],
 ['airborne', 'division', 'of', 'the'],
 ['division', 'of', 'the', 'united'],
 ['of', 'the', 'united', 'states'],
 ['the', 'united', 'states', 'army'],
 ['united', 'states', 'army', 'stationed'],
 ['states', 'army', 'stationed', 'at'],
 ['army', 'stationed', 'at', 'fort'],
 ['stationed', 'at', 'fort', 'campbell'],
 ['at', 'fort', 'campbell', 'kentucky'],
 ['fort', 'campbell', 'kentucky', 'calpernia'],
 ['campbell', 'kentucky', 'calpernia', 'works'],
 ['kentucky', 'calpernia', 'works', 'as'],
 ['calpernia', 'works', 'as', 'a'],
 ['works', 'as', 'a', 'showgirl'],
 ['as', 'a', 'showgirl', 'at'],
 ['a', 'showgirl', 'at', 'a'],
 ['showgirl', 'at', 'a', 'transgender'],
 ['at', 'a', 'transgender', 'revue'],
 ['a', 'transgender', 'revue', 'in'],
 

In [118]:
# create inputs and targets (x and y)
x = []
y = []

for s in test:
    x.append(" ".join(s[:-1]))
    y.append(" ".join(s[1:]))

In [119]:
x[2],y[2]

('a private with', 'private with the')

In [120]:
# create integer-to-token mapping
int2token = {}
cnt = 0

for w in set(" ".join(movie_plots).split()):
    int2token[cnt] = w
    cnt+= 1

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

token2int["the"], int2token[5566]

(5289, 'fancey')

In [121]:
len(token2int)

16592

In [124]:
def get_integer_seq(seq):
    return [token2int[w] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

# convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

In [127]:
x_int[1],y_int[1]

(array([13967,  7357, 16363]), array([ 7357, 16363,  4027]))

In [128]:
len(x_int)

153644

In [129]:
split_input_target(test[2])

('a private with', 'private with the')

In [130]:
def get_batches(arr_x, arr_y, batch_size):
         
    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
        x = arr_x[prv:n,:]
        y = arr_y[prv:n,:]
        prv = n
        yield x, y

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [81]:
class WordLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(lstm_output)
        
        #out = out.contiguous().view(-1, self.n_hidden) 
        out = out.reshape(-1, self.n_hidden) 

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [87]:
# instantiate the model
net = WordLSTM()

# push the model to GPU (avoid it if you are not using the GPU)
#net.cuda()

print(net)

WordLSTM(
  (emb_layer): Embedding(16592, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=16592, bias=True)
)


In [102]:
def train(net, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()
    
    # push model to GPU
    #net.cuda()
    
    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            # push tensors to GPU
            #inputs, targets = inputs, targets

            # detach hidden states
            h = tuple([each.data for each in h])
            inputs  = inputs.type(torch.LongTensor)
            targets  = targets.type(torch.LongTensor)

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()            
            
            if counter % print_every == 0:
            
              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))

In [131]:
train(net, batch_size = 32, epochs=10, print_every=2)

Epoch: 1/10... Step: 2...
Epoch: 1/10... Step: 4...
Epoch: 1/10... Step: 6...
Epoch: 1/10... Step: 8...
Epoch: 1/10... Step: 10...
Epoch: 1/10... Step: 12...
Epoch: 1/10... Step: 14...
Epoch: 1/10... Step: 16...
Epoch: 1/10... Step: 18...
Epoch: 1/10... Step: 20...
Epoch: 1/10... Step: 22...
Epoch: 1/10... Step: 24...
Epoch: 1/10... Step: 26...
Epoch: 1/10... Step: 28...
Epoch: 1/10... Step: 30...
Epoch: 1/10... Step: 32...
Epoch: 1/10... Step: 34...
Epoch: 1/10... Step: 36...
Epoch: 1/10... Step: 38...
Epoch: 1/10... Step: 40...
Epoch: 1/10... Step: 42...
Epoch: 1/10... Step: 44...
Epoch: 1/10... Step: 46...
Epoch: 1/10... Step: 48...
Epoch: 1/10... Step: 50...
Epoch: 1/10... Step: 52...
Epoch: 1/10... Step: 54...
Epoch: 1/10... Step: 56...
Epoch: 1/10... Step: 58...
Epoch: 1/10... Step: 60...
Epoch: 1/10... Step: 62...
Epoch: 1/10... Step: 64...
Epoch: 1/10... Step: 66...
Epoch: 1/10... Step: 68...
Epoch: 1/10... Step: 70...
Epoch: 1/10... Step: 72...
Epoch: 1/10... Step: 74...
Epoch

Epoch: 1/10... Step: 592...
Epoch: 1/10... Step: 594...
Epoch: 1/10... Step: 596...
Epoch: 1/10... Step: 598...
Epoch: 1/10... Step: 600...
Epoch: 1/10... Step: 602...
Epoch: 1/10... Step: 604...
Epoch: 1/10... Step: 606...
Epoch: 1/10... Step: 608...
Epoch: 1/10... Step: 610...
Epoch: 1/10... Step: 612...
Epoch: 1/10... Step: 614...
Epoch: 1/10... Step: 616...
Epoch: 1/10... Step: 618...
Epoch: 1/10... Step: 620...
Epoch: 1/10... Step: 622...
Epoch: 1/10... Step: 624...
Epoch: 1/10... Step: 626...
Epoch: 1/10... Step: 628...
Epoch: 1/10... Step: 630...
Epoch: 1/10... Step: 632...
Epoch: 1/10... Step: 634...
Epoch: 1/10... Step: 636...
Epoch: 1/10... Step: 638...
Epoch: 1/10... Step: 640...
Epoch: 1/10... Step: 642...
Epoch: 1/10... Step: 644...
Epoch: 1/10... Step: 646...
Epoch: 1/10... Step: 648...
Epoch: 1/10... Step: 650...
Epoch: 1/10... Step: 652...
Epoch: 1/10... Step: 654...
Epoch: 1/10... Step: 656...
Epoch: 1/10... Step: 658...
Epoch: 1/10... Step: 660...
Epoch: 1/10... Step:

Epoch: 1/10... Step: 1172...
Epoch: 1/10... Step: 1174...
Epoch: 1/10... Step: 1176...
Epoch: 1/10... Step: 1178...
Epoch: 1/10... Step: 1180...
Epoch: 1/10... Step: 1182...
Epoch: 1/10... Step: 1184...
Epoch: 1/10... Step: 1186...
Epoch: 1/10... Step: 1188...
Epoch: 1/10... Step: 1190...
Epoch: 1/10... Step: 1192...
Epoch: 1/10... Step: 1194...
Epoch: 1/10... Step: 1196...
Epoch: 1/10... Step: 1198...
Epoch: 1/10... Step: 1200...
Epoch: 1/10... Step: 1202...
Epoch: 1/10... Step: 1204...
Epoch: 1/10... Step: 1206...
Epoch: 1/10... Step: 1208...
Epoch: 1/10... Step: 1210...
Epoch: 1/10... Step: 1212...
Epoch: 1/10... Step: 1214...
Epoch: 1/10... Step: 1216...
Epoch: 1/10... Step: 1218...
Epoch: 1/10... Step: 1220...
Epoch: 1/10... Step: 1222...
Epoch: 1/10... Step: 1224...
Epoch: 1/10... Step: 1226...
Epoch: 1/10... Step: 1228...
Epoch: 1/10... Step: 1230...
Epoch: 1/10... Step: 1232...
Epoch: 1/10... Step: 1234...
Epoch: 1/10... Step: 1236...
Epoch: 1/10... Step: 1238...
Epoch: 1/10...

Epoch: 1/10... Step: 1738...
Epoch: 1/10... Step: 1740...
Epoch: 1/10... Step: 1742...
Epoch: 1/10... Step: 1744...
Epoch: 1/10... Step: 1746...
Epoch: 1/10... Step: 1748...
Epoch: 1/10... Step: 1750...
Epoch: 1/10... Step: 1752...
Epoch: 1/10... Step: 1754...
Epoch: 1/10... Step: 1756...
Epoch: 1/10... Step: 1758...
Epoch: 1/10... Step: 1760...
Epoch: 1/10... Step: 1762...
Epoch: 1/10... Step: 1764...
Epoch: 1/10... Step: 1766...
Epoch: 1/10... Step: 1768...
Epoch: 1/10... Step: 1770...
Epoch: 1/10... Step: 1772...
Epoch: 1/10... Step: 1774...
Epoch: 1/10... Step: 1776...
Epoch: 1/10... Step: 1778...
Epoch: 1/10... Step: 1780...
Epoch: 1/10... Step: 1782...
Epoch: 1/10... Step: 1784...
Epoch: 1/10... Step: 1786...
Epoch: 1/10... Step: 1788...
Epoch: 1/10... Step: 1790...
Epoch: 1/10... Step: 1792...
Epoch: 1/10... Step: 1794...
Epoch: 1/10... Step: 1796...
Epoch: 1/10... Step: 1798...
Epoch: 1/10... Step: 1800...
Epoch: 1/10... Step: 1802...
Epoch: 1/10... Step: 1804...
Epoch: 1/10...

Epoch: 1/10... Step: 2304...
Epoch: 1/10... Step: 2306...
Epoch: 1/10... Step: 2308...
Epoch: 1/10... Step: 2310...
Epoch: 1/10... Step: 2312...
Epoch: 1/10... Step: 2314...
Epoch: 1/10... Step: 2316...
Epoch: 1/10... Step: 2318...
Epoch: 1/10... Step: 2320...
Epoch: 1/10... Step: 2322...
Epoch: 1/10... Step: 2324...
Epoch: 1/10... Step: 2326...
Epoch: 1/10... Step: 2328...
Epoch: 1/10... Step: 2330...
Epoch: 1/10... Step: 2332...
Epoch: 1/10... Step: 2334...
Epoch: 1/10... Step: 2336...
Epoch: 1/10... Step: 2338...
Epoch: 1/10... Step: 2340...
Epoch: 1/10... Step: 2342...
Epoch: 1/10... Step: 2344...
Epoch: 1/10... Step: 2346...
Epoch: 1/10... Step: 2348...
Epoch: 1/10... Step: 2350...
Epoch: 1/10... Step: 2352...
Epoch: 1/10... Step: 2354...
Epoch: 1/10... Step: 2356...
Epoch: 1/10... Step: 2358...
Epoch: 1/10... Step: 2360...
Epoch: 1/10... Step: 2362...
Epoch: 1/10... Step: 2364...
Epoch: 1/10... Step: 2366...
Epoch: 1/10... Step: 2368...
Epoch: 1/10... Step: 2370...
Epoch: 1/10...

...

Epoch: 10/10... Step: 45754...
Epoch: 10/10... Step: 45756...
Epoch: 10/10... Step: 45758...
Epoch: 10/10... Step: 45760...
Epoch: 10/10... Step: 45762...
Epoch: 10/10... Step: 45764...
Epoch: 10/10... Step: 45766...
Epoch: 10/10... Step: 45768...
Epoch: 10/10... Step: 45770...
Epoch: 10/10... Step: 45772...
Epoch: 10/10... Step: 45774...
Epoch: 10/10... Step: 45776...
Epoch: 10/10... Step: 45778...
Epoch: 10/10... Step: 45780...
Epoch: 10/10... Step: 45782...
Epoch: 10/10... Step: 45784...
Epoch: 10/10... Step: 45786...
Epoch: 10/10... Step: 45788...
Epoch: 10/10... Step: 45790...
Epoch: 10/10... Step: 45792...
Epoch: 10/10... Step: 45794...
Epoch: 10/10... Step: 45796...
Epoch: 10/10... Step: 45798...
Epoch: 10/10... Step: 45800...
Epoch: 10/10... Step: 45802...
Epoch: 10/10... Step: 45804...
Epoch: 10/10... Step: 45806...
Epoch: 10/10... Step: 45808...
Epoch: 10/10... Step: 45810...
Epoch: 10/10... Step: 45812...
Epoch: 10/10... Step: 45814...
Epoch: 10/10... Step: 45816...
Epoch: 1

Epoch: 10/10... Step: 46284...
Epoch: 10/10... Step: 46286...
Epoch: 10/10... Step: 46288...
Epoch: 10/10... Step: 46290...
Epoch: 10/10... Step: 46292...
Epoch: 10/10... Step: 46294...
Epoch: 10/10... Step: 46296...
Epoch: 10/10... Step: 46298...
Epoch: 10/10... Step: 46300...
Epoch: 10/10... Step: 46302...
Epoch: 10/10... Step: 46304...
Epoch: 10/10... Step: 46306...
Epoch: 10/10... Step: 46308...
Epoch: 10/10... Step: 46310...
Epoch: 10/10... Step: 46312...
Epoch: 10/10... Step: 46314...
Epoch: 10/10... Step: 46316...
Epoch: 10/10... Step: 46318...
Epoch: 10/10... Step: 46320...
Epoch: 10/10... Step: 46322...
Epoch: 10/10... Step: 46324...
Epoch: 10/10... Step: 46326...
Epoch: 10/10... Step: 46328...
Epoch: 10/10... Step: 46330...
Epoch: 10/10... Step: 46332...
Epoch: 10/10... Step: 46334...
Epoch: 10/10... Step: 46336...
Epoch: 10/10... Step: 46338...
Epoch: 10/10... Step: 46340...
Epoch: 10/10... Step: 46342...
Epoch: 10/10... Step: 46344...
Epoch: 10/10... Step: 46346...
Epoch: 1

Epoch: 10/10... Step: 46814...
Epoch: 10/10... Step: 46816...
Epoch: 10/10... Step: 46818...
Epoch: 10/10... Step: 46820...
Epoch: 10/10... Step: 46822...
Epoch: 10/10... Step: 46824...
Epoch: 10/10... Step: 46826...
Epoch: 10/10... Step: 46828...
Epoch: 10/10... Step: 46830...
Epoch: 10/10... Step: 46832...
Epoch: 10/10... Step: 46834...
Epoch: 10/10... Step: 46836...
Epoch: 10/10... Step: 46838...
Epoch: 10/10... Step: 46840...
Epoch: 10/10... Step: 46842...
Epoch: 10/10... Step: 46844...
Epoch: 10/10... Step: 46846...
Epoch: 10/10... Step: 46848...
Epoch: 10/10... Step: 46850...
Epoch: 10/10... Step: 46852...
Epoch: 10/10... Step: 46854...
Epoch: 10/10... Step: 46856...
Epoch: 10/10... Step: 46858...
Epoch: 10/10... Step: 46860...
Epoch: 10/10... Step: 46862...
Epoch: 10/10... Step: 46864...
Epoch: 10/10... Step: 46866...
Epoch: 10/10... Step: 46868...
Epoch: 10/10... Step: 46870...
Epoch: 10/10... Step: 46872...
Epoch: 10/10... Step: 46874...
Epoch: 10/10... Step: 46876...
Epoch: 1

Epoch: 10/10... Step: 47344...
Epoch: 10/10... Step: 47346...
Epoch: 10/10... Step: 47348...
Epoch: 10/10... Step: 47350...
Epoch: 10/10... Step: 47352...
Epoch: 10/10... Step: 47354...
Epoch: 10/10... Step: 47356...
Epoch: 10/10... Step: 47358...
Epoch: 10/10... Step: 47360...
Epoch: 10/10... Step: 47362...
Epoch: 10/10... Step: 47364...
Epoch: 10/10... Step: 47366...
Epoch: 10/10... Step: 47368...
Epoch: 10/10... Step: 47370...
Epoch: 10/10... Step: 47372...
Epoch: 10/10... Step: 47374...
Epoch: 10/10... Step: 47376...
Epoch: 10/10... Step: 47378...
Epoch: 10/10... Step: 47380...
Epoch: 10/10... Step: 47382...
Epoch: 10/10... Step: 47384...
Epoch: 10/10... Step: 47386...
Epoch: 10/10... Step: 47388...
Epoch: 10/10... Step: 47390...
Epoch: 10/10... Step: 47392...
Epoch: 10/10... Step: 47394...
Epoch: 10/10... Step: 47396...
Epoch: 10/10... Step: 47398...
Epoch: 10/10... Step: 47400...
Epoch: 10/10... Step: 47402...
Epoch: 10/10... Step: 47404...
Epoch: 10/10... Step: 47406...
Epoch: 1

Epoch: 10/10... Step: 47874...
Epoch: 10/10... Step: 47876...
Epoch: 10/10... Step: 47878...
Epoch: 10/10... Step: 47880...
Epoch: 10/10... Step: 47882...
Epoch: 10/10... Step: 47884...
Epoch: 10/10... Step: 47886...
Epoch: 10/10... Step: 47888...
Epoch: 10/10... Step: 47890...
Epoch: 10/10... Step: 47892...
Epoch: 10/10... Step: 47894...
Epoch: 10/10... Step: 47896...
Epoch: 10/10... Step: 47898...
Epoch: 10/10... Step: 47900...
Epoch: 10/10... Step: 47902...
Epoch: 10/10... Step: 47904...
Epoch: 10/10... Step: 47906...
Epoch: 10/10... Step: 47908...
Epoch: 10/10... Step: 47910...
Epoch: 10/10... Step: 47912...
Epoch: 10/10... Step: 47914...
Epoch: 10/10... Step: 47916...
Epoch: 10/10... Step: 47918...
Epoch: 10/10... Step: 47920...
Epoch: 10/10... Step: 47922...
Epoch: 10/10... Step: 47924...
Epoch: 10/10... Step: 47926...
Epoch: 10/10... Step: 47928...
Epoch: 10/10... Step: 47930...
Epoch: 10/10... Step: 47932...
Epoch: 10/10... Step: 47934...
Epoch: 10/10... Step: 47936...
Epoch: 1

In [133]:
# predict next token
def predict(net, tkn, h=None):

    # tensor inputs
    x = np.array([[token2int[tkn]]])
    inputs = torch.from_numpy(x)

    # push to GPU
    inputs = inputs.type(torch.LongTensor)

    # detach hidden state from history
    h = tuple([each.data for each in h])

    # get the output of the model
    out, h = net(inputs, h)

    # get the token probabilities
    p = F.softmax(out, dim=1).data

    p = p.cpu()

    p = p.numpy()
    p = p.reshape(p.shape[1],)

    # get indices of top 3 values
    top_n_idx = p.argsort()[-3:][::-1]

    # randomly select one of the three indices
    sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

    # return the encoded value of the predicted char and the hidden state
    return int2token[sampled_token_index], h


# function to generate text
def sample(net, size, prime='it is'):
        
    # push to GPU
    #net.cuda()
    
    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
        token, h = predict(net, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(net, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)


In [145]:
import joblib
joblib.dump(net,'nlg_model')

['nlg_model']

In [149]:
joblib.dump(token2int,'token2int')

['token2int']

In [151]:
joblib.dump(int2token,'int2token')

['int2token']

In [150]:
token2int1 = joblib.load('token2int')

In [146]:
net1 = joblib.load('nlg_model')

In [152]:
int2token1 = joblib.load('int2token')

In [147]:
sample(net1, 15)

'it is not able that she was not a vampire of a vampire to be the train'

In [148]:
sample(net1, 10, prime = "yo yo ")

'yo yo has not quit a new vampire of the passengers to'